<a href="https://colab.research.google.com/github/kocielnik/rule_one_stocks/blob/main/rule_one_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%%file mos.py

from yfinance import Ticker


class Deal:

    """
    >>> repr(Deal("tsla", 1, 1))
    'Deal("tsla", 1, 1)'
    """

    def __init__(
        self, symbol, sticker_price, price
    ):
        self.symbol = symbol
        self.sticker_price = sticker_price
        self.price = price
        self.percent_of_sticker_price = price / sticker_price * 100

    def __repr__(self):
        return f'Deal("{self.symbol}", {self.sticker_price}, {self.price})'

    def __eq__(self, other):
        if self.symbol != other.symbol:
            return False

        if self.sticker_price != other.sticker_price:
            return False

        if self.price != other.price:
            return False

        return True

class MockTicker:
    def __init__(self, _):
        self.info = {
            "currentPrice": 352.5
        }


def get_price(symbol, Ticker=Ticker):
    """
    >>> get_price("msft", Ticker=MockTicker)
    352.5
    >>> get_price([])
    {}
    >>> get_price(["msft"], Ticker=MockTicker)
    {'msft': 352.5}
    """
    if type(symbol) == str:
        price = Ticker(symbol).info["currentPrice"]
        return price

    prices = {
        the_symbol: get_price(the_symbol, Ticker=Ticker)
        for the_symbol in symbol
    }
    return prices


def get_deal(sticker_prices, Ticker=Ticker):
    deals = []

    for symbol in sticker_prices:
        sticker_price = sticker_prices[symbol]
        price = get_price(symbol, Ticker=Ticker)
        deals.append(Deal(symbol, sticker_price, price))
    
    if len(sticker_prices) == 1:
        return deals[0]
    
    return deals

def test_get_deal_one():
    sticker_prices = {
        "msft": 118
    }

    deal = get_deal(sticker_prices, Ticker=MockTicker)

    assert deal == Deal("msft", 118, 352.5)

def test_get_deal_multiple():
    sticker_prices = {
        "msft": 118,
        "tsm": 22
    }

    deals = get_deal(sticker_prices, Ticker=MockTicker)

    assert deals[0] == Deal("msft", 118, 352.5)

Overwriting mos.py


In [24]:
!pytest --doctest-modules mos.py

============================= test session starts ==============================
platform linux -- Python 3.10.11, pytest-7.2.2, pluggy-1.0.0
rootdir: /content
plugins: anyio-3.6.2
collected 4 items                                                              

mos.py ....                                                              [100%]

============================== 4 passed in 0.97s ===============================
